In [3]:
import pandas as pd
from pandas import DataFrame
import re

In [4]:
from pandas import Series

base_forms = pd.read_csv('base_forms.csv')
base_forms = base_forms.drop(index=0)
base_forms = base_forms.reset_index(drop=True)
base_forms.head()


,p-3,p-2,p-1,p0a,p0b,p+1,p+2,p+3,p+4,stem transitivity,meaning
0,ʌ,NaN,NaN,der,NaN,den,NaN,NaN,NaN,TR,make (sb) urite
1,ʌ,NaN,NaN,der,NaN,den,i,NaN,NaN,NTR,be made urinate
2,ʌ,NaN,NaN,gard,NaN,NaN,NaN,NaN,NaN,NTR,return
3,ʌ,NaN,NaN,gard,NaN,den,NaN,NaN,NaN,TR,"return, make (sb) return"
4,ʌ,NaN,NaN,gard,NaN,den,i,NaN,NaN,NTR,be returned


In [7]:
class Positions:
	PM3 = 'p-3'
	P0A = 'p0a'
	P0B = 'p0b'
	PP1 = 'p+1'
	PP2 = 'p+2'
	PP4 = 'p+4'
	TRANS = 'stem transitivity'
	MEANING = 'meaning'


class Morpheme:
	NULL = '∅'
	A = '-ʌ'
	EM = '-em'
	I = '-i'
	E = '-e'
	IE = '-iɛ'
	OM = '-om'
	ENDE = '-endɛ'
	ME = 'me-'


class Person:
	FS = 'first person singular'
	SS = 'second person singular'
	TSM = 'third person singular masculine'
	TSF = 'third person singular feminine'
	FP = 'first person plural'
	SP = 'second person plural'
	TP = 'third person plural'

	@classmethod
	def to_list(cls):
		return [cls.FS, cls.SS, cls.TSM, cls.TSF, cls.FP, cls.SP, cls.TP]



class Modality:
	IMP = 'imperative'
	SBJ = 'subjunctive'
	IND = 'indicative'

	@classmethod
	def to_list(cls):
		return [cls.IMP, cls.SBJ, cls.IND]


class Aspect:
	IPF = 'imperfective'
	PRF = 'perfect'
	PLU = 'pluperfect'
	UN = 'unmarked'

	@classmethod
	def to_list(cls):
		return [cls.IPF, cls.PRF, cls.PLU, cls.UN]


class Polarity:
	NEG = 'negative'
	AFF = 'affirmative'


class InvalidInflectionError(Exception):
	def __init__(self, message = 'inflection does not exist'):
		self.message = message
		super().__init__(self, self.message)


class InflectorBase:
	preverb: str
	imperfective: str
	negation: str
	stem_neutral: str
	stem_past: str
	causative: str
	passive: str
	past: str
	agreement: str

	aspect: str
	modality: str
	polarity: str
	person: str

	row: Series

	def __init__(self,
	             row: Series,
	             modality: str,
				 person: str,
	             aspect: str,
	             polarity = ''):
		self.row = row
		self.modality = modality
		self.person = person
		self.aspect = aspect
		self.polarity = polarity
		self.imperfective = ''
		self.agreement = ''


	"""append preverb- in template (P-3)."""
	def set_preverb(self):
		self.preverb = self.row[Positions.PM3]

	"""append stem-neutral (P0A) under Stem"""
	def set_stem_neutral(self):
		self.stem_neutral = self.row[Positions.P0A]

	"""append -den if filled in the verb list, under Causative (PP1)"""
	def set_causative(self):
		self.causative = self.row[Positions.PP1]

	"""append -i if filled in in the verb list under Passive (PP2)"""
	def set_passive(self):
		self.passive = self.row[Positions.PP2]



class PresentInflector(InflectorBase):
	__tense = 'present'

	"""
        If P+2 is not occupied & mod is imperative:
            Insert AGR in p+4: ∅ for 2s/ -ʌ for 2p
        elif modality is chosen as subjunctive or indicative:
            Insert -em for 1s/ -i for 2s/ -e for 3sm/ -iɛ for 3sf/ -om for 1p/ -ʌ for 2p, -endɛ for 3p
        else:
            IF modality is indictative and aspect is imperfective:
                Insert me- in P-2
                If p-3 is filled with be-/de-, delete be/-de
    """

	def set_agreement(self):
		if self.row[Positions.PP2] is None and self.modality == Modality.IMP:
			if self.person == Person.SS:
				self.agreement = Morpheme.NULL
			elif self.person == Person.SP:
				self.agreement = Morpheme.A
			else:
				raise InvalidInflectionError()
		elif self.modality == Modality.SBJ or self.modality == Modality.IND:
			if self.person == Person.FS:
				self.agreement = Morpheme.EM
			elif self.person == Person.SS:
				self.agreement = Morpheme.I
			elif self.person == Person.TSM:
				self.agreement = Morpheme.E
			elif self.person == Person.TSF:
				self.agreement = Morpheme.IE
			elif self.person == Person.FP:
				self.agreement = Morpheme.OM
			elif self.person == Person.SP:
				self.agreement = Morpheme.A
			elif self.person == Person.TP:
				self.agreement = Morpheme.ENDE
		else:
			if self.modality == Modality.IND and self.aspect == Aspect.IPF:
				self.imperfective = Morpheme.ME
				self.preverb =  Morpheme.NULL if bool(re.match('(be-)|(de-)', self.preverb)) else self.preverb


	def build(self):
		self.set_preverb()
		self.set_stem_neutral()
		self.set_causative()
		self.set_passive()
		self.set_agreement()
		return self


	def view(self):
		return f"{self.preverb}-{self.imperfective}-{self.stem_neutral}-{self.causative}-{self.passive}-{self.agreement}"

In [9]:
verbs_df = pd.DataFrame(columns=[
	'preverb',
	'imperfective',
	'stem_neutral',
	'causative',
	'passive',
	'agreement',
	'modality',
	'person',
	'aspect'
])

for index, row in base_forms.iterrows():
	for modality in Modality.to_list():
		for person in Person.to_list():
			for aspect in Aspect.to_list():
				try:
					inflector = PresentInflector(row, modality, person, aspect)\
						.build()
					verbs_df = verbs_df.append({
						'preverb': inflector.preverb,
						'imperfective': inflector.imperfective,
						'stem_neutral': inflector.stem_neutral,
						'causative': inflector.causative,
						'passive': inflector.passive,
						'agreement': inflector.agreement,
						'modality': modality,
						'person': person,
						'aspect': aspect
					}, ignore_index=True)
				except InvalidInflectionError as error:
					print(error.message)
					continue

verbs_df.head()


,preverb,imperfective,stem_neutral,causative,passive,agreement,modality,person,aspect
0,ʌ,,der,den,NaN,,imperative,first person singular,imperfective
1,ʌ,,der,den,NaN,,imperative,first person singular,perfect
2,ʌ,,der,den,NaN,,imperative,first person singular,pluperfect
3,ʌ,,der,den,NaN,,imperative,first person singular,unmarked
4,ʌ,,der,den,NaN,,imperative,second person singular,imperfective


In [ ]:
class A:
	test = 'stupid'

class B(A):
	def print(self):
		print(self.test)

B().print()